# the `teeth.array` module: explore and tokenize text


say something.

In [ ]:
raw_text_path = 'moby_dick.txt'

with open( raw_text_path, 'r') as f:
        raw_text = f.read()

In [ ]:
raw_text[ 0 : 27 ]